In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import HashingTF, Tokenizer, VectorAssembler

In [ ]:
import os
pwd = os.getcwd()
exploration_abs_path = f"{pwd}/exploration.ipynb"
exploration_abs_path

In [ ]:
%run {pwd + "/exploration.ipynb"}

In [ ]:
spark = create_spark_session("Spark_Application_Name")
all_datasets = []
for f in ['AMAZON.csv', 'APPLE.csv', 'FACEBOOK.csv', 'GOOGLE.csv',
            'MICROSOFT.csv', 'TESLA.csv', 'ZOOM.csv']:
    print(f"{f}:")
    df = load_data(spark, 'stocks_data/' + f)
    all_datasets.append(df)

Input:  
    - 4xHigh  
    - 4xOpen  
    - 4xClose  
    - 4xLow  
    - RSI (not implemented)  
    - Volatilité  
    - Momentum  
    - Upside/Downside  
    - IMI  
    - MFI  
  
  
régression polynomiale  
régression quantile  
réseau de neurones récurrent  

In [ ]:
from pyspark.sql.window import Window

df_with_lag = df
for i in range(1, 4):
    for s in ["High", "Low", "Open", "Close"]:
        df_with_lag = df_with_lag.withColumn(s + str(i), F.lag(F.col(s), i).over(Window.orderBy('Date')))

In [ ]:
assembler = VectorAssembler(
    inputCols=["High", "Low", "Open", "Close", "Volume"], outputCol="features")
lr = LinearRegression()
pipeline = Pipeline(stages=[assembler, lr])

In [ ]:
import pyspark.sql.functions as F
# isnan, when, count, datediff, mean, lag, 
# col, month, year, weekofyear, date_format, dayofmonth, avg, stddev, desc, sum
df_labeled = df.withColumn("label", F.abs(df.High - df.Low) )

In [ ]:
# Fit the pipeline to training documents.
model = pipeline.fit(df_labeled)

In [ ]:
predDF = model.transform(df)

In [ ]:
predDF.show()